In [1]:
def companion(f): # companion matrix of 
    R = f.parent()
    K = R.base_ring()
    n = f.degree()
    if f.leading_coefficient() != 1:
        raise ValueError("Polynomial must be monic")
    coeffs = [f[i] for i in range(n)]
    M = Matrix(K, n, n)
    for i in range(n - 1):
        M[i, i + 1] = 1
    for j in range(n):
        M[n - 1, j] = -coeffs[j]
    return M

In [2]:
def block_companion(f, x=1):
    # block jordan companion
    C = companion(f)
    K = C.base_ring()
    n = C.nrows()
    size = x * n
    J = Matrix(K, size, size)
    for i in range(x):
        J[n*i : n*(i+1), n*i : n*(i+1)] = C
        if i < x - 1:
            J[n*i : n*(i+1), n*(i+1) : n*(i+2)] = identity_matrix(K, n)
    return J

In [3]:
def Q(q, d, s):
    return (1 - 1/(q**d))**2

In [4]:
print(float(Q(7, 2, 1)*Q(7, 1, 3)))

0.7050123672959396


In [5]:
def build_matrix(poly_config, inv_facts):
    blocks = []
    for inv_fact in inv_facts:
        for i in range(len(poly_config)):
            if inv_fact[i] != 0:
                blocks.append(block_companion(poly_config[i],inv_fact[i]))
    return block_diagonal_matrix(blocks)

In [6]:
q = 7
R = PolynomialRing(GF(q), 'x')
x = R.gen()

poly_config = [x^2 + 3*x + 6, x + 4] # check irreducible over field, or generate
inv_facts = [[2, 1]] # {fg, g, g}
def theoretical(poly_config, inv_facts):
    prob = 1
    for i in range(len(poly_config)):
        d = 0 # max degree
        s = 0
        for fact in inv_facts: 
            if fact[i] > d: 
                d, s = fact[i], 1
            elif fact[i] == d:
                s += 1
        prob *= Q(q, d, s)
    return prob
print(float(theoretical(poly_config, inv_facts)))

0.7050123672959396
